# Criação de sistema de recomendação de filmes

#### https://www.kaggle.com/code/alyssonbispopereira/recomenda-o-de-filmes-ptbr/data

In [2]:
#importando bibliotecas
import pandas as pd
import numpy as np

In [3]:
#importando a base de dados com os filmes
filmes = pd.read_csv('movies_metadata.csv', low_memory=False)
filmes.head(4)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0


In [4]:
#importando a base de dados de avaliação
avaliacoes = pd.read_csv('ratings.csv')
avaliacoes.head(4)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546


# Fazer pré processamento dos dados

In [5]:
# filtrando as colunas que serão usadas
# renomeando as variaveis

filmes = filmes [['id','original_title','original_language','vote_count']]

# renomeando
filmes.rename(columns = {'id':'ID_FILME','original_title':'TITULO','original_language':'LINGUAGEM','vote_count':'QT_AVALIACOES'}, inplace = True)

#exibindo
filmes.head(4)

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0


In [6]:
# filtrando as variaveis que serão usadas
avaliacoes = avaliacoes [['userId','movieId','rating']]

# renomeando as variaveis 
avaliacoes.rename(columns = {'userId':'ID_USUARIO','movieId':'ID_FILME','rating':'AVALIACAO'}, inplace = True)

avaliacoes.head(4)

,ID_USUARIO,ID_FILME,AVALIACAO
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0


In [7]:
# verificando se existe valores nulos
filmes.isna().sum()

ID_FILME          0
TITULO            0
LINGUAGEM        11
QT_AVALIACOES     6
dtype: int64

In [8]:
# como são poucos valores faltantes, podemos remover
filmes.dropna(inplace=True)

In [9]:
# verificando se os valores nulos foram removidos
filmes.isna().sum()

ID_FILME         0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [10]:
# verificando se existe valores nulos em 'avaliacoes'
avaliacoes.isna().sum()

ID_USUARIO    0
ID_FILME      0
AVALIACAO     0
dtype: int64

In [11]:
#observando a quantidade de avaliacoes por usuarios
avaliacoes['ID_USUARIO'].value_counts()

45811     18276
8659       9279
270123     7638
179792     7515
228291     7410
          ...  
30155         1
9641          1
164717        1
243426        1
234625        1
Name: ID_USUARIO, Length: 270896, dtype: int64

In [12]:
# pegando apenas os usuarios que fizeram assima de 999 avaliacoes
qt_avaliacoes = avaliacoes['ID_USUARIO'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

(2509,)

In [13]:
# observando os usuarios selecionados
y

Int64Index([ 45811,   8659, 270123, 179792, 228291, 243443,  98415, 229879,
             98787, 172224,
            ...
            227649, 244253, 257117,  30733, 196384,  53075, 220764, 214328,
             14354, 182812],
           dtype='int64', length=2509)

In [14]:
#observando o tamanho do dataset 'avaliacoes'
avaliacoes.shape

(26024289, 3)

In [15]:
# pegando os usuarios que avaliaram assima de 999 vezes
avaliacoes = avaliacoes[avaliacoes['ID_USUARIO'].isin(y)]

In [16]:
# observando tamanho do dataset avaliacoes
avaliacoes.shape

(3844582, 3)

In [17]:
#observando o dataframe avaliacoes
avaliacoes.head()

,ID_USUARIO,ID_FILME,AVALIACAO
17291,229,1,3.0
17292,229,2,3.0
17293,229,4,2.0
17294,229,5,1.0
17295,229,7,2.0


In [18]:
#observando o dataframe filmes
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


In [19]:
# separando somente os filmes que tiveram assima de 999 avaliacoes
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

In [20]:
# separando quantidade de filmes por linguagem
filmes_linguagem = filmes['LINGUAGEM'].value_counts()
filmes_linguagem.head(20)

en    1100
fr       5
ja       5
it       3
ko       2
pt       1
de       1
es       1
cn       1
sv       1
id       1
Name: LINGUAGEM, dtype: int64

In [21]:
# filtrando apenas filmes em inglês(en)
filmes = filmes[filmes['LINGUAGEM'] == 'en']

In [22]:
#observando os tipos das variaveis
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   object 
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB


In [23]:
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3844582 entries, 17291 to 26023521
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID_USUARIO  int64  
 1   ID_FILME    int64  
 2   AVALIACAO   float64
dtypes: float64(1), int64(2)
memory usage: 117.3 MB


In [24]:
# modificando a variavel id_filme para int para unir os dataframes
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

In [25]:
# observando se o tipo da coluna mudou 
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   int32  
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), int32(1), object(2)
memory usage: 38.7+ KB


In [26]:
# concatenando os DataFrames(unindo)
avaliacoes_e_filmes = avaliacoes.merge(filmes, on = 'ID_FILME')
avaliacoes_e_filmes.head()

,ID_USUARIO,ID_FILME,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
0,229,12,1.0,Finding Nemo,en,6292.0
1,741,12,3.0,Finding Nemo,en,6292.0
2,1932,12,0.5,Finding Nemo,en,6292.0
3,3437,12,2.0,Finding Nemo,en,6292.0
4,3694,12,1.5,Finding Nemo,en,6292.0


In [27]:
# observando quantidade de filmes com avaliacoes pelo tamnho do arquivo
avaliacoes_e_filmes.shape

(189882, 6)

In [28]:
#observando se ficol valor nulo
avaliacoes_e_filmes.isna().sum()

ID_USUARIO       0
ID_FILME         0
AVALIACAO        0
TITULO           0
LINGUAGEM        0
QT_AVALIACOES    0
dtype: int64

In [29]:
# descartando valores duplicados
avaliacoes_e_filmes.drop_duplicates(['ID_USUARIO','ID_FILME'], inplace = True)

In [31]:
#observando a quantidade de registros
avaliacoes_e_filmes.shape

(189882, 6)

In [34]:
# excluindo variavel ID_FILMES porque não sera utilizada
del avaliacoes_e_filmes['ID_FILME']

In [35]:
#observando o DataFrame
avaliacoes_e_filmes.head()

,ID_USUARIO,AVALIACAO,TITULO,LINGUAGEM,QT_AVALIACOES
0,229,1.0,Finding Nemo,en,6292.0
1,741,3.0,Finding Nemo,en,6292.0
2,1932,0.5,Finding Nemo,en,6292.0
3,3437,2.0,Finding Nemo,en,6292.0
4,3694,1.5,Finding Nemo,en,6292.0


In [36]:
# Fazendo pivot
# transformando cada id_usuario em uma variavel
filmes_pivot = avaliacoes_e_filmes.pivot_table(columns = 'ID_USUARIO', index = 'TITULO', values = 'AVALIACAO')

#observando os dados
filmes_pivot.head()

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.5,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN
12 Angry Men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
127 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.5,2.0,NaN,NaN,NaN,NaN
2 Fast 2 Furious,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#preenchendo todos os valores nulos com 0
filmes_pivot.fillna(0, inplace = True)
filmes_pivot.head()

ID_USUARIO,229,231,741,836,1104,1136,1243,1380,1652,1846,...,269632,269750,269913,270071,270123,270213,270237,270564,270654,270887
TITULO,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.5,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
127 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.5,2.0,0.0,0.0,0.0,0.0
2 Fast 2 Furious,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# criando matriz sparsa
from scipy.sparse import csr_matrix

#tranformando do DataFrame em matriz sparsa
filmes_sparse = csr_matrix(filmes_pivot)

In [40]:
#observando o tipo
type(filmes_sparse)

scipy.sparse._csr.csr_matrix

In [41]:
#importando KNN
from sklearn.neighbors import NearestNeighbors

In [42]:
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)

NearestNeighbors(algorithm='brute')

# Prevendo as sugestões de filmes

In [43]:
#Kneighbors - visinhos mais proximos(alguma semelhança)
# 127 Hours
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['127 Hours'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]])  

Index(['127 Hours', 'American Hustle', 'The Expendables 2', 'Lord of War',
       'RED 2'],
      dtype='object', name='TITULO')


In [44]:
# Toy Story
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['Toy Story'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 

Index(['Toy Story', 'Meet the Fockers', 'Top Gun',
       'Harry Potter and the Chamber of Secrets',
       'Austin Powers: International Man of Mystery'],
      dtype='object', name='TITULO')


In [45]:
# 1408
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['1408'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 

Index(['1408', 'Pirates of the Caribbean: At World's End', 'Platoon', 'Snitch',
       'The Expendables 2'],
      dtype='object', name='TITULO')


In [46]:
# 2 Fast 2 Furious
distances, sugestions = modelo.kneighbors(filmes_pivot.filter(items = ['2 Fast 2 Furious'], axis=0).values.reshape(1, -1))

for i in range(len(sugestions)):
    print(filmes_pivot.index[sugestions[i]]) 

Index(['2 Fast 2 Furious', 'Bambi', 'The Matrix Reloaded',
       'Brokeback Mountain', 'Lord of War'],
      dtype='object', name='TITULO')
